In [1]:
## To work with the data
import pandas as pd, numpy as np

In [2]:
## Let's read our data
df = pd.read_csv("consumer_compliants.zip", compression = "zip")

In [3]:
## Take a loot at the data
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,This auto loan was opened on XX/XX/2020 in XXX...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX of 2019 I noticed a debt for {$620.00}...,None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX an...",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in...",None,CAPITAL ONE FINANCIAL CORPORATION,CA,None,None,Consent provided,Web,3/6/2020,Closed with explanation,Yes,NaN,3556237
4,2/14/2020,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,This debt was incurred due to medical malpract...,Company believes it acted appropriately as aut...,"Merchants and Professional Bureau, Inc.",OH,432XX,None,Consent provided,Web,2/14/2020,Closed with explanation,Yes,NaN,3531704


In [4]:
## To be abe to read whole narrative
pd.set_option("display.max_colwidth", -1)

<ipython-input-4-1b7976ed63e7>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


- there are a lot of attributes which we are not interested in for the task in hand. We'll be working on Customer complaint narrative and try to perform topic modeling on that.

In [5]:
## Useful columns
df = df[['Product', 'Consumer complaint narrative']]
df.columns = ['Product', 'Narrative']

In [6]:
df.head()

,Product,Narrative
0,Vehicle loan or lease,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account."
1,Debt collection,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back."
2,Vehicle loan or lease,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times."
3,Checking or savings account,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that."
4,Debt collection,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never stopped it from being reported when I specifically told them not to, due to the circumstances above."


In [7]:
df.shape

(57453, 2)

In [8]:
df['Product'].value_counts()
## Class imbalance

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                       9799 
Checking or savings account    7003 
Student loan                   2950 
Vehicle loan or lease          2736 
Name: Product, dtype: int64

#### Preprocessing

In [9]:
## performes tokenization.
from nltk import word_tokenize

In [10]:
## Custom Tokenization:
## Those card numbers with Xx in them are of no use. also, I am not interested in amount and other numerical features. 
def tokenize(sentance):
    tokens = word_tokenize(sentance)
    tokens = [word for word in tokens if ( word.isalpha() and len(word)>3 and len(word.strip('Xx/'))>2) ]
    return tokens

In [11]:
## Used for splitting our data into train and test set.
from sklearn.model_selection import train_test_split

X, y = df[['Narrative']], df[['Product']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

In [12]:
## Resolving class Imbalance
from imblearn.over_sampling import RandomOverSampler

strategy = {
    "Debt collection"              :  21772,
    "Credit card or prepaid card"  :  13193,
    "Mortgage"                     :  11000 ,
    "Checking or savings account"  :  9000 ,
    "Student loan"                 :  7000 ,
    "Vehicle loan or lease"        :  7000 }

ros = RandomOverSampler(sampling_strategy = strategy, random_state=42)
X_train, y_train = ros.fit_resample(X_train, y_train)

c:\users\raj-pc\appdata\local\programs\python\python38\lib\site-packages\imblearn\utils\_validation.py:318: UserWarning: After over-sampling, the number of samples (21772) in class Debt collection will be larger than the number of samples in the majority class (class #Debt collection -> 17418)
  warnings.warn(


In [13]:
## we need to represent words as numerical values/ vectors.
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
## Very helpful indeed. Different libraries may have differet implementation of the same functionality.
## It is useful to take a look at the parameters of the function
TfidfVectorizer?

In [16]:
vectorizer = TfidfVectorizer(tokenizer = tokenize, stop_words = 'english', 
                            max_df = 0.75, min_df = 50, max_features = 10000)
vectors = vectorizer.fit_transform(X_train['Narrative'])

In [17]:
vectors.shape

(68965, 4954)

In [18]:
## Feature names mapped to the array index.
## 1st feature in vectors is the word at 1 index of the list.
vectorizer.get_feature_names()

['aadvantage',
 'abandoned',
 'abide',
 'abiding',
 'ability',
 'able',
 'abroad',
 'abruptly',
 'absence',
 'absent',
 'absolute',
 'absolutely',
 'absurd',
 'abuse',
 'abused',
 'abusing',
 'abusive',
 'accelerate',
 'accelerated',
 'accent',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'access',
 'accessed',
 'accessible',
 'accessing',
 'accident',
 'accidentally',
 'accommodate',
 'accompanied',
 'accomplish',
 'accomplished',
 'accord',
 'accordance',
 'according',
 'accordingly',
 'account',
 'accountability',
 'accountable',
 'accountant',
 'accounted',
 'accounting',
 'accounts',
 'accredited',
 'accrual',
 'accrue',
 'accrued',
 'accrues',
 'accruing',
 'acct',
 'accumulate',
 'accumulated',
 'accumulating',
 'accuracy',
 'accurate',
 'accurately',
 'accusations',
 'accused',
 'accusing',
 'achieve',
 'acknowledge',
 'acknowledged',
 'acknowledgement',
 'acknowledges',
 'acknowledging',
 'acknowledgment',
 'acount',
 'acquiescence',
 'acquire',
 'acqui

### Model Building

In [19]:
from sklearn.decomposition import LatentDirichletAllocation

In [19]:
LatentDirichletAllocation?

In [20]:
lda = LatentDirichletAllocation(n_components = 6, learning_method = 'online', max_iter = 100,
                         n_jobs = -1, random_state = 123)

W1 = lda.fit_transform(vectors)

H1 = lda.components_

In [21]:
W1

array([[0.02208511, 0.14860288, 0.7636007 , 0.02192789, 0.02195088,
        0.02183253],
       [0.68393709, 0.06282135, 0.06416959, 0.06317694, 0.06308944,
        0.06280559],
       [0.02771513, 0.02765738, 0.02770626, 0.02767087, 0.8616083 ,
        0.02764206],
       ...,
       [0.0243226 , 0.02433498, 0.8787929 , 0.02430773, 0.02413233,
        0.02410946],
       [0.03758623, 0.09856799, 0.75132845, 0.0375955 , 0.03746531,
        0.03745653],
       [0.03098156, 0.09106048, 0.73618989, 0.0309848 , 0.08006459,
        0.03071868]])

In [22]:
## W1 -> Probability of document being of the topic(i). 6 topics->6probabilities.
## H1 -> Topic->Words
print(W1.shape, H1.shape)

(68965, 6) (6, 4954)


In [23]:
vocab = np.array(vectorizer.get_feature_names())

## Gives last (numwords-1) numbers in descending order. from n to 0.
## So, for each topic, we get top 10 words that appear in that topic.
def get_top_words(words):
    res = []
    for index in np.argsort(words)[:-10:-1]:
        res.append(vocab[index])
    return res
## top_words = lambda words: [vocab[index] for index in np.argsort(words)[:-top:-1]]

## Get topic words using H1.
topic_words = (get_top_words(words) for words in H1)
topics = [','.join(t) for t in topic_words]

In [24]:
topics
## These are based on my intuition. Can be interpreted entirely differently as well.
## 0 -> Debt Colelction
## 1 -> Student Loans
## 2 -> Credit card or prepaid card
## 3 -> Vehicle loan or lease
## 4 -> Checking or savings account
## 5 -> Mortgage

['debt,credit,collection,report,account,company,reporting,information,letter',
 'loan,loans,navient,student,payments,forbearance,modification,mortgage,school',
 'account,payment,bank,card,credit,told,called,loan,time',
 'wells,fargo,calls,calling,phone,number,debt,called,stop',
 'identity,theft,victim,belong,report,affidavit,account,does,attached',
 'citi,usaa,apartment,credits,gift,transworld,complex,thats,rent']

In [25]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
df1 = pd.DataFrame(np.round(W1, 2), columns = colnames, index = X_train.index)
significant_topic = np.argmax(df1.values, axis = 1)
df1["Dominant Topic"] = significant_topic

In [26]:
df1 = X_train.join(df1)
df1 = df1.join(y_train)

In [27]:
df1.head()

,Narrative,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Dominant Topic,Product
0,"Good morning. As it is known worldwide, there is a COVID-19 Pandemic going on. I requested forbearance for my student loans ( which should of been stayed automatically ) and to my surprise, I received an automatic debit from AES in my account where no funds are available plus I requested a forbearance via AES 's website. \n\nI am being charged fees for overdraft and will pursue this issue as AES/PHEE and/or any servicer that has managed this account has caused great damage, harm and other damages. \n\nStop auto debit as CARES and and Families First act requieres, pay damages and return fees. Plus their is another complaint regarding illegal fees charged which is not closed and am awaiting for CFPB to inform about appeals and legal process and or guidance as how to proceed. \n\nThank You XXXX XXXX XXXX",0.02,0.15,0.76,0.02,0.02,0.02,2,Student loan
1,XX/XX/XXXX for the amount of $XXXX then for the amount $XXXX for collection lass time it update wasXX/XX/XXXX,0.68,0.06,0.06,0.06,0.06,0.06,0,Debt collection
2,CAINE & WEINER ( 01 XXXX XXXX ) is a result of identity theft. I recently discovered through my new monitoring service this account being reported I made efforts to remove this account from my file as the amount of {$130.00} does not belong to me. This account was fraudulently opened through this company in XXXX of 2016 an has been reported to the FTC and authorities as Identity theft. I have submitted all of my information needed to resolve this matter in this complaint and through the FTC report attached and do not feel comfortable giving any further information in any way to any others at this time due to the further possibilities of identity theft.,0.03,0.03,0.03,0.03,0.86,0.03,4,Debt collection
3,"XXXX XXXX XXXX XXXX XXXX XXXX XXXX, PA XXXX To Whom It May Concern, This letter is a formal complaint that you are reporting inaccurate and incomplete credit information on my Credit Report. \nCollection Creditor : LVNV FUNDING LLC Original creditor name XXXX XXXX XXXX XXXX XXXX XXXX Amount : {$750.00} ( Reason for investigation : Not Mine. Please Un-verify & delete this information ) Collection Creditor : XXXX XXXX XXXX Original creditor name : XXXX XXXX XXXX XXXX XXXX XXXX Amount : {$720.00} ( Reason for investigation : Not Mine. Please Un-verify & delete this information ) Creditor : XXXX XXXX XXXX ( Reason for investigation : Not Mine. Please Un-verify & delete this information ) Creditor : XXXX XXXX XXXX ( Reason for investigation : Not Mine. Please Un-verifXXXX & delete this information ) Creditor : XXXX XXXX XXXX ( Reason for investigation : Not Mine. Please Un-verify & delete this information ) Creditor : XXXX XXXX XXXX ( Reason for investigation : Not Mine. Please Un-verify & delete this information ) Creditor : XXXX XXXX XXXX XXXX XXXX XXXX ( Reason for investigation : Not Mine. Please Un-verify & delete this information ) Creditor : XXXX XXXX ( Reason for investigation : Not Mine. Please Un-verify & delete this information ) I understand that mistakes happen but your inaccurate information is costing me money in higher interest rates and I have enough expenses as is. \n\nThank You Sincerely yours,",0.64,0.04,0.19,0.05,0.04,0.04,0,Debt collection
4,"I can not give specific dates as I fell on hard times but I was repeatedly contacted by this company to the point of harassment. I've had to block upwards of 10 numbers to get relief, I was having so much anxiety whenever my phone would ring I debated changing my number all together.",0.03,0.03,0.44,0.43,0.03,0.03,2,Debt collection


In [28]:
W2 = lda.transform(vectorizer.transform(X_test['Narrative']))

X_test["Dominant Topic"] = np.argmax(W2, axis =1)

<ipython-input-28-5ef669510ec2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["Dominant Topic"] = np.argmax(W2, axis =1)


In [29]:
X_test.join(y_test).head(10)

Narrative  \
21957  I purchased a new vehicle and financed it with a loan from USAA. I was confused about the specifics of a 0.5 % APR discount if I went to a dealer who particiapted in the USAA car buying program. I contacted the bank by phone to confirm the details of the specific discount. I confirmed verbally on a recorded line prior to purchasing my vehicle that I only needed to contact a auto dealer, verify that the dealer participated in the car buying program with USAA, then purchase a vehicle from that dealer and the 0.5 % APR discount would be applied to the rate of my loan. USAA has no physical bank branch locations so I had to rely on a verbal confirmation on my phone call. \n\nI went and purchased my vehicle in mid XX/XX/XXXX. I then received a APR billing form noting that I was being charged without the 0.5 % APR discount applied. \n\nI called USAA on Thursday XX/XX/XXXX approximately XXXX XXXX  until XXXX to attempt to sort it out. \n\nThe bank let me talk to an executive dispute resolution person named XXXX. XXXX reviewed the recorded call when I called USAA to clarify the details of the program and he admitted to me that USAA had made an error and incorrectly explained the details that I needed to follow to qualify for the loan. He explained that I was misled and that USAA would fix the issue for me. He took some time to confirm the details with his boss before coming back on the line with me. He explained that USAA would require me to pay the same fixed amount for a monthly payment but they would reduce the APR with the rate that I was told I would qualify for which was ~2.8ish % not the 3.3ish % that I was incorrectly charged. \n\nHe explained that I would continue to pay my fixed monthly amount but that the principle of the loan would be paid off earlier thus saving me the money I should have gotten because I would have paid off the loan earlier and saved the difference in interest that I was owed. \n\nHe explained that I should receive a letter in mid XX/XX/XXXX detailing the specifics of this remedy. I specifically asked about the detailed amount I would save and that this I specifically asked that they would reduce the total loan interest rate and that they provide me written details of this remedy. XXXX confirmed to me on the phone that they would honor the 0.5 % decrease in my rate because the person I had talked to prior at USAA had misspoke and given me incorrect information. XXXX did explain the details of the USAA car buying program and what I would have to do to qualify for the discount in the future. \n\nUltimately this resolution would shorten the life of the loan since I will be paying the old monthly payment at the 3.3ish percent over the life of the loan but the loan amount will remain the same. I did confirm that I would still be able to pay off the loan early still. \n\nI waited until XX/XX/XXXX after I got a letter from USAA rejecting XXXX XXXX   internal request. I contacted USAA and a representative told me that it was a sort of clerical error and that they would resubmit the paperwork through the proper channels. I stated to them that I was working on the assumption that what XXXX told me still stands on XXXX XXXX and USAA will have to figure out how to make it work for me. \n\nI received an email decline notice on or around XX/XX/XXXX from USAA. I called and spoke to a representative at USAA again and was told they would not honor the previous statements. I asked to speak to an executive resolutions specialist and was sent to talk with XXXX from their executive resolutions team. XXXX and I spoke and he reviewed the previous phone calls and notes in their system. He explained he would advocate for me and get the issue taken care of for me. He said he would resubmit and get back to me. \n\nXXXX and I talked again around XX/XX/XXXX, he explained it was not working and he was excalating to the car buying service and I should expect to hear back. \n\nOn XX/XX/XXXX I received word that XXXX 